In [3]:
import pandas as pd
import numpy as np

In [4]:
from __future__ import print_function

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.data_utils import get_file

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
data=pd.read_csv('C:\\Users\\Administrator\\Related questions\\Dataset\\train.csv')
test_set=pd.read_csv('C:\\Users\\Administrator\\Related questions\\Dataset\\test.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [7]:
#converting the elements of the train and test question series to string
data['question1'] = data['question1'].apply(lambda x: (str(x)))
data['question2'] = data['question2'].apply(lambda x: (str(x)))


test_set['question1'] = test_set['question1'].apply(lambda x: (str(x)))
test_set['question2'] = test_set['question2'].apply(lambda x: (str(x)))

In [8]:
#target contains the is-duplicate flag series
target=data['is_duplicate']

In [9]:
#make lists out of the question1 and question2 series in the train and test dataset
question1 = list(data['question1'])
question2 = list(data['question2'])

test1=list(test_set['question1'])
test2=list(test_set['question2'])

In [10]:
#length of train and test dataset (length of question1 = length of question 2)
print(len(question1))
print(len(test1))

404290
3563475


In [11]:
#creating tokenizer, consider most frequent 200000 unique words
tokenizer = Tokenizer(num_words=200000)
#fit the tokenizer with the lists created
tokenizer.fit_on_texts(question1+question2+test1+test2)

In [12]:
#generating tokens for the unique words in training dataset
question1_word_sequences = tokenizer.texts_to_sequences(question1)
question2_word_sequences = tokenizer.texts_to_sequences(question2)

#dictionary of unique words in corpus (training and test sets) as key and token as value
word_index = tokenizer.word_index 

print("Words in index: %d" % len(word_index))

Words in index: 137042


In [13]:
#generating tokens for the unique words in test dataset
test1_word_sequences = tokenizer.texts_to_sequences(test1)
test2_word_sequences = tokenizer.texts_to_sequences(test2)

In [14]:
#padding token sequences to make the length equal to 25 (truncation if necessary)
q1_data = pad_sequences(question1_word_sequences, maxlen=25)
q2_data = pad_sequences(question2_word_sequences, maxlen=25)
test1_data=pad_sequences(test1_word_sequences, maxlen=25)
test2_data=pad_sequences(test2_word_sequences, maxlen=25)

#numpy array of the target value (is_duplicate  flag)
labels = np.array(target, dtype=int)

#shape gives the dimensions
print('Shape of question1 data tensor:', q1_data.shape)
print('Shape of question2 data tensor:', q2_data.shape)
print('Shape of label tensor:', labels.shape)

Shape of question1 data tensor: (404290, 25)
Shape of question2 data tensor: (404290, 25)
Shape of label tensor: (404290,)


In [15]:
#create empty dictionary
embeddings_index = {}

#open GLoVe file
f = open('glove.840B.300d.txt', encoding="utf-8")

## Eg of a line in GLoVe file
# the -0.7865 0.675 ... +288 other weights

In [16]:
#preprocessing 
for line in f:
    if line[0] == line[1] == ' ':
        word = ' '
        parts = [' '] + line.strip().split(' ')
    else: 
        parts = line.split(' ')
        #the word itself ('the')
        word = parts[0]
    #weights representing the relatedness with 300 other words (-0.7865 0.675 ...)
    coefs = np.asarray(parts[1:])
    #creating items of the dictionary
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 1280544 word vectors.


In [17]:
#create numpy array with zeros
embedding_matrix = np.zeros((len(word_index) + 1, 300))

#word contains the word and i contains the weights
for word, i in word_index.items():
    #retieve weights
    embedding_vector = embeddings_index.get(word)
    #if there are weights associated with the word, 
    if embedding_vector is not None:
        # words not found in embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [18]:
embedding_matrix.shape

(137043, 300)

In [19]:
#save in numpy files for easy reading the second time
np.save(open('q1_train.npy', 'wb'), q1_data)
np.save(open('q2_train.npy', 'wb'), q2_data)
np.save(open('test1.npy', 'wb'), test1_data)
np.save(open('test2.npy', 'wb'), test2_data)

np.save(open('label_train.npy', 'wb'), labels)
np.save(open('word_embedding_matrix.npy', 'wb'), embedding_matrix)

In [1]:
%store word_index

UsageError: Unknown variable 'word_index'
